# Rock Paper Scissors

### Import the used modules and definition of some constants

In [1]:
import numpy as np
import cvxpy as cp
import tabulate

ROCK, PAPER, SCISSORS = 0, 1, 2

### Definition of some utilities functions

In [2]:
# Used to make the text bold
def boldText(text):
    return "\033[1m" + text + "\033[0m"

# Used to display the payoff matrix
def displayPayoffMatrix(M, mode):
    print(f'The payof matrix of the {mode} game is:')
    header = ['', 'Rock', 'Paper', 'Scissors']
    body = [['Rock', M[ROCK, ROCK], M[ROCK, PAPER], M[ROCK, SCISSORS]],
            ['Paper', M[PAPER, ROCK], M[PAPER, PAPER], M[PAPER, SCISSORS]],
            ['Scissors', M[SCISSORS, ROCK], M[SCISSORS, PAPER], M[SCISSORS, SCISSORS]]]
    print(tabulate.tabulate(body, header, tablefmt="fancy_grid"))

# Used to display the optimal strategy and expected payoff
def displayResults(player, probabilities, expectedPayoff):
    print(f'The Nash strategy for the {boldText(player)} player is:')
    header = ['Move', 'Probability']
    body = [['Rock', round(float(probabilities[ROCK]), 3)],
            ['Paper', round(float(probabilities[PAPER]), 3)],
            ['Scissors', round(float(probabilities[SCISSORS]), 3)]]
    print(tabulate.tabulate(body, header, tablefmt="fancy_grid"))
    print(f'The expected payoff for the {boldText(player)} player is: {round(expectedPayoff, 3)}')
    

## 3.1.1 Traditional Rock Paper Scissors

### Payoff matrix definition

In [3]:
M = np.array([[0, -1, 1], [1, 0, -1], [-1, 1, 0]])
displayPayoffMatrix(M, 'traditional rock paper scissors')

The payof matrix of the traditional rock paper scissors game is:
╒══════════╤════════╤═════════╤════════════╕
│          │   Rock │   Paper │   Scissors │
╞══════════╪════════╪═════════╪════════════╡
│ Rock     │      0 │      -1 │          1 │
├──────────┼────────┼─────────┼────────────┤
│ Paper    │      1 │       0 │         -1 │
├──────────┼────────┼─────────┼────────────┤
│ Scissors │     -1 │       1 │          0 │
╘══════════╧════════╧═════════╧════════════╛


### Assuming that the row-player’s strategy is to play rock with probability 1, derive the best-response strategy of the column-player 

In [4]:
# Define the variables
x = cp.Variable(3, nonneg=True)

# Define the objective function
obj = cp.Minimize(cp.sum(cp.multiply(M[ROCK,:], x)))

# Define the constraints
constraints = [cp.sum(x) == 1]

# Define the problem
prob = cp.Problem(obj, constraints)

# Solve the problem
prob.solve()

# Display the results
displayResults('column', x.value, -prob.value)

The Nash strategy for the column player is:
╒══════════╤═══════════════╕
│ Move     │   Probability │
╞══════════╪═══════════════╡
│ Rock     │             0 │
├──────────┼───────────────┤
│ Paper    │             1 │
├──────────┼───────────────┤
│ Scissors │             0 │
╘══════════╧═══════════════╛
The expected payoff for the column player is: 1.0


### Construct the Nash strategies of both players and report the expected payoff of the row-player. Interpret your results.

In [5]:
# Define the variables
p = cp.Variable((3,1), nonneg=True)
a = cp.Variable((1), nonneg=False)

# Define the objective function
obj = cp.Maximize(a)

# Define the constraints
constraints = [cp.sum(p) == 1, a <= (p.T@M).T]

# Define the problem
prob = cp.Problem(obj, constraints)

# Solve the problem
prob.solve()
print(f'Status: {prob.status}\n')

column_player_values = constraints[1].dual_value

# Display the results
displayResults('row', p.value, prob.value)
print('')
displayResults('column', column_player_values, -prob.value)

Status: optimal

The Nash strategy for the row player is:
╒══════════╤═══════════════╕
│ Move     │   Probability │
╞══════════╪═══════════════╡
│ Rock     │         0.333 │
├──────────┼───────────────┤
│ Paper    │         0.333 │
├──────────┼───────────────┤
│ Scissors │         0.333 │
╘══════════╧═══════════════╛
The expected payoff for the row player is: -0.0

The Nash strategy for the column player is:
╒══════════╤═══════════════╕
│ Move     │   Probability │
╞══════════╪═══════════════╡
│ Rock     │         0.333 │
├──────────┼───────────────┤
│ Paper    │         0.333 │
├──────────┼───────────────┤
│ Scissors │         0.333 │
╘══════════╧═══════════════╛
The expected payoff for the column player is: 0.0


# 3.1.2 Modified Rock Paper Scissors
In this case we are going to consider a modified version of the rock-paper-scissors game where the payoff of the row-player amount to +2 instead of +1 if she wins by playing rock

### Payoff matrix definition

In [6]:
M = np.array([[0, -1, 2], [1, 0, -1], [-1, 1, 0]])

# Plot the M matrix as a table
displayPayoffMatrix(M, 'modified rock paper scissors')

The payof matrix of the modified rock paper scissors game is:
╒══════════╤════════╤═════════╤════════════╕
│          │   Rock │   Paper │   Scissors │
╞══════════╪════════╪═════════╪════════════╡
│ Rock     │      0 │      -1 │          2 │
├──────────┼────────┼─────────┼────────────┤
│ Paper    │      1 │       0 │         -1 │
├──────────┼────────┼─────────┼────────────┤
│ Scissors │     -1 │       1 │          0 │
╘══════════╧════════╧═════════╧════════════╛


### Problem definition and results

In [7]:
# Define the variables
p = cp.Variable((3,1), nonneg=True)
a = cp.Variable((1), nonneg=False)

# Define the objective function

obj = cp.Maximize(a)

# Define the constraints
constraints = [cp.sum(p) == 1, a <= (p.T@M).T]

# Define the problem
prob = cp.Problem(obj, constraints)

# Solve the problem
prob.solve()
print(f'Status: {prob.status}\n')

column_player_values = constraints[1].dual_value

# Display the results
displayResults('row', p.value, prob.value)
print('')
displayResults('column', column_player_values, -prob.value)

Status: optimal

The Nash strategy for the row player is:
╒══════════╤═══════════════╕
│ Move     │   Probability │
╞══════════╪═══════════════╡
│ Rock     │         0.25  │
├──────────┼───────────────┤
│ Paper    │         0.417 │
├──────────┼───────────────┤
│ Scissors │         0.333 │
╘══════════╧═══════════════╛
The expected payoff for the row player is: 0.083

The Nash strategy for the column player is:
╒══════════╤═══════════════╕
│ Move     │   Probability │
╞══════════╪═══════════════╡
│ Rock     │         0.333 │
├──────────┼───────────────┤
│ Paper    │         0.417 │
├──────────┼───────────────┤
│ Scissors │         0.25  │
╘══════════╧═══════════════╛
The expected payoff for the column player is: -0.083
